> Algumas questões foram alteradas depois do envio na Udemy

##### Parte 1 > Realizando import's e iniciando pyspark

In [15]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.types import StringType
import random
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from functools import reduce

In [2]:
# Nesta etapa, adicoine código para ler o arquivo nomes_aleatorios.txt através do comando spark.read.csv. 
# Carregue-o para dentro de um dataframe chamado df_nomes e, por fim, liste algumas linhas através do método show. Exemplo: df_nomes.show(5)

spark = SparkSession.builder.master("local[*]").appName("Tarefa").getOrCreate()
df_nomes = spark.read.csv('../Tarefa3/nomes_aleatorios.txt')
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



##### Parte 2 > Renomear coluna, imprimir o schema e mostrar 10 linhas

In [3]:
df_nomes = df_nomes.withColumnRenamed('_c0', 'Nomes')
df_nomes.printSchema()
df_nomes.show(10)


root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



##### Parte 3 > Adicionar nova coluna chamada Escolaridade

In [4]:
escolaridade = ['Fundamental', 'Médio', 'Superior']

df_nomes = df_nomes.withColumn('Escolaridade', udf(f=lambda: escolaridade[random.randint(0,2)],returnType=StringType())())

##### Parte 4 > Adicionar países da América do Sul


In [5]:
paises = ["Brasil", "Argentina", "Colômbia", "Peru", "Venezuela", "Chile", "Equador",
            "Bolívia", "Paraguai", "Uruguai", "Guiana", "Suriname", "Guiana Francesa"]

df_nomes = df_nomes.withColumn('Pais', udf(f=lambda: paises[random.randint(0,12)],returnType=StringType())())

##### Parte 5 > Adicione nova coluna chamada AnoNascimento

In [6]:
df_nomes = df_nomes.withColumn("AnoNascimento", expr("CAST(FLOOR(RAND() * (66) + 1945) AS INT)"))

##### Parte 6 > Usando o método select do dataframe (df_nomes), selecione as pessoas que nasceram neste século

In [7]:
df_select = df_nomes.select('*').where(df_nomes.AnoNascimento >= 2000)
df_select.show(10)

+----------------+------------+---------------+-------------+
|           Nomes|Escolaridade|           Pais|AnoNascimento|
+----------------+------------+---------------+-------------+
|Donald Golightly|    Superior|      Venezuela|         2009|
|   Brian Farrell|       Médio|Guiana Francesa|         2001|
|    Leroy Strahl|    Superior|        Equador|         2009|
|    Ernest Hulet|       Médio|        Bolívia|         2010|
|        Mary Lee| Fundamental|        Equador|         2008|
|      Daryl Page|       Médio|         Brasil|         2003|
|   Ricky Gilbert|    Superior|       Colômbia|         2002|
|Flossie Anderson|       Médio|           Peru|         2004|
|  Ashley Trosper| Fundamental|       Colômbia|         2010|
|     Milton Rowe| Fundamental|      Argentina|         2003|
+----------------+------------+---------------+-------------+
only showing top 10 rows



##### Parte 7 > Usando Spark SQL repita o processo da Pergunta 6.

In [8]:
df_nomes.createOrReplaceTempView('pessoas')
spark.sql('SELECT * FROM pessoas WHERE AnoNascimento >= 2000').show(10)


+----------------+------------+---------------+-------------+
|           Nomes|Escolaridade|           Pais|AnoNascimento|
+----------------+------------+---------------+-------------+
|Donald Golightly|       Médio|Guiana Francesa|         2009|
|   Brian Farrell| Fundamental|        Uruguai|         2001|
|    Leroy Strahl|       Médio|       Paraguai|         2009|
|    Ernest Hulet|       Médio|         Brasil|         2010|
|        Mary Lee| Fundamental|      Argentina|         2008|
|      Daryl Page|       Médio|      Venezuela|         2003|
|   Ricky Gilbert| Fundamental|        Equador|         2002|
|Flossie Anderson| Fundamental|Guiana Francesa|         2004|
|  Ashley Trosper|       Médio|          Chile|         2010|
|     Milton Rowe|    Superior|        Uruguai|         2003|
+----------------+------------+---------------+-------------+
only showing top 10 rows



##### Parte 8 > Conte o número de pessoas que são da geração Millennials


In [9]:
m = df_nomes.select('*').where(df_nomes.AnoNascimento.between(1980, 1994)).count()
print(m)


2272990


##### Parte 9 > Repita o processo da Pergunta 8 utilizando Spark SQL

In [10]:
spark.sql('SELECT COUNT(*) as Contagem FROM pessoas WHERE AnoNascimento BETWEEN 1980 AND 1994').show()

+--------+
|Contagem|
+--------+
| 2272990|
+--------+



##### Parte 10 > Obtenha a quantidade de pessoas de cada país para uma das gerações

In [11]:
sqlContext = SQLContext(spark)

babys = sqlContext.sql("SELECT Pais, 'Baby Boomers' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1944 AND 1964\
                        GROUP BY Pais")

ger_x = sqlContext.sql("SELECT Pais, 'Geração X' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1965 AND 1979\
                        GROUP BY Pais")

millenials = sqlContext.sql("SELECT Pais, 'Millennials' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1980 AND 1994\
                        GROUP BY Pais")

ger_z = sqlContext.sql("SELECT Pais, 'Geração Z' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1995 AND 2015\
                        GROUP BY Pais")

C:\Users\mathe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [12]:
geracoes = [babys, ger_x, millenials, ger_z]
geracao_union = reduce(DataFrame.unionAll, geracoes)

In [21]:
geracao_union.orderBy(asc("Pais"),"Geracao", asc("Quantidade")).show(geracao_union.count(), False)

+---------------+------------+----------+
|Pais           |geracao     |Quantidade|
+---------------+------------+----------+
|Argentina      |Baby Boomers|233723    |
|Argentina      |Geração X   |174004    |
|Argentina      |Geração Z   |187347    |
|Argentina      |Millennials |174264    |
|Bolívia        |Baby Boomers|232410    |
|Bolívia        |Geração X   |174781    |
|Bolívia        |Geração Z   |186241    |
|Bolívia        |Millennials |174456    |
|Brasil         |Baby Boomers|233363    |
|Brasil         |Geração X   |174760    |
|Brasil         |Geração Z   |187128    |
|Brasil         |Millennials |175269    |
|Chile          |Baby Boomers|233180    |
|Chile          |Geração X   |174723    |
|Chile          |Geração Z   |186519    |
|Chile          |Millennials |174811    |
|Colômbia       |Baby Boomers|232813    |
|Colômbia       |Geração X   |175041    |
|Colômbia       |Geração Z   |186926    |
|Colômbia       |Millennials |175110    |
|Equador        |Baby Boomers|2329